In [161]:
import modern_robotics as mr
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()

In [162]:
def exp3(omega, theta):
    omega = sp.skew(omega)
    R = sp.eye(3) + sp.sin(theta) * omega + (1 - sp.cos(theta)) * omega * omega
    return R

def Ad(T):
    AdT = sp.zeros(6)
    R = sp.Matrix(T[:3, :3])
    AdT[:3, :3] = R
    AdT[3:, 3:] = R
    AdT[3:, :3] = sp.skew(T[:3, 3]) * R
    return AdT

def rotX(alfa_im1):
    Rx = sp.eye(4)
    Rx[1,1] =    sp.cos(alfa_im1)
    Rx[1,2] =   -sp.sin(alfa_im1)
    Rx[2,1] =    sp.sin(alfa_im1)
    Rx[2,2] =    sp.cos(alfa_im1)
    return Rx

def rotZ(alfa_im1):
    Rz = sp.eye(4)
    Rz[0,0] =    sp.cos(alfa_im1)
    Rz[0,1] =   -sp.sin(alfa_im1)
    Rz[1,0] =    sp.sin(alfa_im1)
    Rz[1,1] =    sp.cos(alfa_im1)
    return Rz

def transX(a_im1):
    trA = sp.eye(4)
    trA[0,3] =  a_im1
    return trA

def transZ(d_i):
    trA = sp.eye(4)
    trA[2,3] =  d_i
    return trA


In [163]:
# Oppg 1
print("Task 1:")

th1, th2, th3, th4, th5, th6 = dynamicsymbols('theta_1, theta_2, theta_3, theta_4, theta_5, theta_6')

config = sp.Matrix([[0,0,0,th1],[sp.pi/2,0,0,th2],[0,455,0,th3 + sp.pi/2],[sp.pi/2, 35, 420, th4],[sp.pi/2,0,0,th5],[sp.pi/2, 0, -80,th6]])
config

Task 1:


⎡0   0    0     θ₁  ⎤
⎢                   ⎥
⎢π                  ⎥
⎢─   0    0     θ₂  ⎥
⎢2                  ⎥
⎢                   ⎥
⎢                  π⎥
⎢0  455   0   θ₃ + ─⎥
⎢                  2⎥
⎢                   ⎥
⎢π                  ⎥
⎢─  35   420    θ₄  ⎥
⎢2                  ⎥
⎢                   ⎥
⎢π                  ⎥
⎢─   0    0     θ₅  ⎥
⎢2                  ⎥
⎢                   ⎥
⎢π                  ⎥
⎢─   0   -80    θ₆  ⎥
⎣2                  ⎦

In [164]:
# oppg 2
Mi = sp.Matrix([[sp.eye(4)]*6])

for i in range(6):
    if i == 1:
        Mi[:,4*i:4*(i+1)] = rotX(config[i,0]) * transX(config[i,1]) * transZ(config[i,2]) * rotZ(sp.pi/2) # We compansate for the rotation of -pi/2 done when finding the D-H parameters 
    else:
        Mi[:,4*i:4*(i+1)] = rotX(config[i,0]) * transX(config[i,1]) * transZ(config[i,2])

M = sp.eye(4)
for n in range(5,-1,-1):
    M = Mi[:,4*n:4*(n+1)] * M

#M = sp.Matrix([[1,0,0,455+420+80],[0,1,0], [0,0,1,-35], [0,0,0,1]])
M 


⎡0  0  -1  500⎤
⎢             ⎥
⎢0  1  0    0 ⎥
⎢             ⎥
⎢1  0  0   490⎥
⎢             ⎥
⎣0  0  0    1 ⎦

In [165]:
# oppg 3
Ai = sp.Matrix([[0,-1,0,0],[1,0,0,0],[0,0,0,0],[0,0,0,0]]) # This is a given matrix due to revolute joints

S_sp = sp.zeros(6)

for i in range(6):
    dot_sum = sp.eye(4)
    for n in range(i,-1,-1):
        dot_sum = Mi[:,4*n:4*(n+1)] * dot_sum
    S_skew = dot_sum * Ai * sp.Inverse(dot_sum)
    S_sp[0,i] = S_skew[2,1] 
    S_sp[1,i] = S_skew[0,2] 
    S_sp[2,i] = S_skew[1,0] 
    S_sp[3,i] = S_skew[0,3] 
    S_sp[4,i] = S_skew[1,3] 
    S_sp[5,i] = S_skew[2,3] 

S_sp

⎡0  0    0    1    0     -1 ⎤
⎢                           ⎥
⎢0  -1  -1    0    1     0  ⎥
⎢                           ⎥
⎢1  0    0    0    0     0  ⎥
⎢                           ⎥
⎢0  0   455   0   -490   0  ⎥
⎢                           ⎥
⎢0  0    0   490   0    -490⎥
⎢                           ⎥
⎣0  0    0    0   420    0  ⎦

In [166]:
# oppg 4, find body frame screw axis, S_bp

M_inv = mr.TransInv(M)          #Finding inverse of M
Ad_M_inv = mr.Adjoint(M_inv)    #Computing [Ad_M^-1]

#Using B_i = [Ad_M^-1]S_i
S_bp = sp.zeros(6,6)
for i in range(6):
     S_bp[:, i] = Ad_M_inv * S_sp[:, i]

S_bp

⎡ 1    0    0   0    0   0⎤
⎢                         ⎥
⎢ 0   -1   -1   0    1   0⎥
⎢                         ⎥
⎢ 0    0    0   -1   0   1⎥
⎢                         ⎥
⎢ 0   500  500  0   -80  0⎥
⎢                         ⎥
⎢500   0    0   0    0   0⎥
⎢                         ⎥
⎣ 0   490  35   0    0   0⎦

## Oppgave 5

In [167]:
import open3d as o3d
import numpy as np

def Ry_sym(theta):
    ct = sp.cos(theta)
    st = sp.sin(theta)
    R = sp.Matrix([[ct, 0.0, st], [0.0, 1.0, 0.0], [-st, 0, ct]])
    return R

class Mesh_object:
    mesh_objects = []
    def __init__(self):
        pass
    
    def set_colour(self, colour):
        self.colour = colour

class Joint(Mesh_object):
    colour = [0,1,0]
    def __init__(self):
        Mesh_object.__init__(self)
        self.joint = o3d.geometry.TriangleMesh.create_cylinder(radius=0.1, height=0.3)
        self.coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
        self.update_mesh_list()
        self.set_colour(self.colour)

    def update_mesh_list(self):
        self.mesh_objects.append(self.joint)
        self.mesh_objects.append(self.coord)
    
    def set_colour(self, colour):
        self.joint.paint_uniform_color(colour)

class Link(Mesh_object):
    colour = [1,0,1]
    def __init__(self, lenght):
        Mesh_object.__init__(self)
        self.lenght = lenght
        self.link = o3d.geometry.TriangleMesh.create_cylinder(radius=0.01, height=self.lenght).rotate(Ry_sym(np.pi/2)).translate(np.array([self.lenght/2,0,0]))
        self.update_mesh_list()
        self.set_colour(self.colour)

    def update_mesh_list(self):
        self.mesh_objects.append(self.link)
    
    def set_colour(self, colour):
        self.link.paint_uniform_color(colour)
f1 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)



In [168]:
J1 = Joint()
L1 = Link(5)
L2 = Link(1)
#o3d.visualization.draw_geometries(Mesh_object.mesh_objects)


In [169]:
def make_mesh_objects(colour):
    f0 = o3d.geometry.TriangleMesh.create_coordinate_frame()
    f1 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)   
    f2 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)  
    f3 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)   
    f4 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
    
    j1 = o3d.geometry.TriangleMesh.create_cylinder(radius=0.1, height=0.3)
    j2 = o3d.geometry.TriangleMesh.create_cylinder(radius=0.1, height=0.3)
    j3 = o3d.geometry.TriangleMesh.create_cylinder(radius=0.1, height=0.3)

    l1 = o3d.geometry.TriangleMesh.create_cylinder(radius=0.01, height=L1).rotate(Ry_sym(np.pi/2)).translate(np.array([L1/2,0,0]))
    l2 = o3d.geometry.TriangleMesh.create_cylinder(radius=0.01, height=L2).rotate(Ry_sym(np.pi/2)).translate(np.array([L2/2,0,0]))
    l3 = o3d.geometry.TriangleMesh.create_cylinder(radius=0.01, height=L3).rotate(Ry_sym(np.pi/2)).translate(np.array([L3/2,0,0]))
    
    meshes = np.array([[f1,f2,f3,f4],
              [j1,j2,j3],
              [l1,l2,l3]])
    
    for e in meshes:
         for i in range(len(e)):
             e[i].paint_uniform_color(colour)

    return meshes

def transform(Si,theta):
    T = mr.MatrixExp6(mr.VecTose3(Si) * theta)

def mesh_transform(meshes, S_list, thetas):
    T_list = np.array(len(thetas)) # riktig lengde?
    for i in range(6, -1, -1):
        dot_sum = 0
        for S in S_list:
            


    for e in meshes:
        for i in range(len(e)):
            e[i].transform(S_list[i])
    return meshes

IndentationError: expected an indented block (Temp/ipykernel_1496/1693744880.py, line 37)